In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tensorflow
import tflearn
import random
import json
import pickle

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
with open('./Datasets/intents.json') as file:
    data = json.load(file)

In [3]:
words = []
labels =[]
docs_x = []
docs_y = []

In [4]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrd = nltk.word_tokenize(pattern)
        words.extend(wrd) # wrds is already a list so rather than appending through it and appending one by one we can append the entire list
        docs_x.append(wrd)
        docs_y.append(intent['tag'])
        if intent['tag'] not in labels:
            labels.append(intent['tag'])
        
words= [stemmer.stem(w.lower()) for w in words if w!= '?']
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty=[ 0 for _ in range(len(labels))]   #classes to labels

for x,doc in enumerate(docs_x):
    bag =[]
    
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if  w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])]=1
    training.append(bag)
    output.append(output_row)

In [5]:
training = np.array(training)
output = np.array(output)

tensorflow.reset_default_graph()

net = tflearn.input_data(shape =[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, len(output[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [6]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
#model = Sequential()
#model.add(Dense(8, input_dim=len(training[0]), activation='relu'))
#model.add(Dense(8, activation='relu'))
#model.add(Dense(len(output[0]), activation='softmax'))

In [8]:
model.fit(training, output,n_epoch=1000, batch_size=8, show_metric=True)
model.save('chatbotV1.tflearn')

Training Step: 4999  | total loss: 0.01192 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.01192 - acc: 1.0000 -- iter: 32/39
Training Step: 5000  | total loss: 0.01189 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.01189 - acc: 1.0000 -- iter: 39/39
--
INFO:tensorflow:C:\Users\aditya\Desktop\Granthium_ChatBot\chatbotV1.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

In [14]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.75:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
        else:
            print("Sorry,I didn't get that could you please try again!")

In [ ]:
chat()

Start talking with the bot (type quit to stop)!
